In [11]:
import nibabel as nib
import os
from nilearn.image import crop_img
from nilearn.image import new_img_like, resample_to_img
import numpy as np

import sys
sys.path.append('..')

from scipy.ndimage.filters import gaussian_filter, gaussian_laplace, laplace, maximum_filter, minimum_filter, prewitt, sobel

from fetal_net.postprocess import postprocess_prediction

%matplotlib

def vod(mask1, mask2, verbose=False):
    mask1, mask2 = mask1.flatten(), mask2.flatten()
    intersection = np.sum((mask1 + mask2) > 1)
    union = np.sum((mask1+mask2) > 0)
    if verbose:
        print('intersection\t', intersection)
        print('union\t\t', union)
    return 1 - (intersection + 1) / (union + 1)

Using matplotlib backend: TkAgg


In [ ]:
!ls ../../Datasets/fetus/78

In [163]:
!ls ../../predictions/unet96_all/predictions/val/

222  260  41_2


In [184]:
src_dir = '../../predictions/unet96_all/predictions/val/'
subject_id = '222'
vol = nib.load(os.path.join(src_dir, subject_id, 'data_volume.nii.gz'))
truth = nib.load(os.path.join(src_dir, subject_id, 'truth.nii.gz'))
pred = nib.load(os.path.join(src_dir, subject_id, 'prediction.nii.gz'))

In [185]:
print('thres\t\t: {}'.format(vod(truth.get_data(), pred.get_data() > 0.5)))
print('pp_0_01\t\t: {}'.format(vod(truth.get_data(), postprocess_prediction(pred.get_data(), gaussian_std=0, threshold=0.1))))
print('pp_05_05\t: {}'.format(vod(truth.get_data(), postprocess_prediction(pred.get_data(), gaussian_std=0.5, threshold=0.5))))
print('pp_1_05\t\t: {}'.format(vod(truth.get_data(), postprocess_prediction(pred.get_data(), gaussian_std=1, threshold=0.5))))

thres		: 0.1412296536763702
pp_0_01		: 0.1259709038930662
pp_05_05	: 0.13838074797935984
pp_1_05		: 0.15213493934900457


In [166]:
np.max(vol.get_data())

4.408514612614356

In [186]:
new_img_like(vol, data=vol.get_data()+pred.get_data()*3).orthoview()

<OrthoSlicer3D: (256, 256, 52)>

Traceback (most recent call last):
  File "/home/galdude33/anaconda3/envs/keras/lib/python3.6/site-packages/matplotlib/cbook/__init__.py", line 388, in process
    proxy(*args, **kwargs)
  File "/home/galdude33/anaconda3/envs/keras/lib/python3.6/site-packages/matplotlib/cbook/__init__.py", line 228, in __call__
    return mtd(*args, **kwargs)
TypeError: _cleanup() takes 1 positional argument but 2 were given


In [99]:
new_img_like(vol, data=pred.get_data()*2).orthoview()

<OrthoSlicer3D: (256, 256, 108)>

Traceback (most recent call last):
  File "/home/galdude33/anaconda3/envs/keras/lib/python3.6/site-packages/matplotlib/cbook/__init__.py", line 388, in process
    proxy(*args, **kwargs)
  File "/home/galdude33/anaconda3/envs/keras/lib/python3.6/site-packages/matplotlib/cbook/__init__.py", line 228, in __call__
    return mtd(*args, **kwargs)
TypeError: _cleanup() takes 1 positional argument but 2 were given


In [9]:
new_img_like(vol, data=truth.get_data()-(postprocess_prediction(pred.get_data(), threshold=0.5, gaussian_std=0.75))).orthoview()

NameError: name 'postprocess_prediction' is not defined

In [40]:
from matplotlib import pyplot as plt
a, bins = np.histogram(pred.get_data())
plt.hist(pred.get_data().reshape([-1]), bins=100) 
plt.title("histogram") 
plt.show()
print(a,bins)

[5162475    5280    3609    3332    4608    4416    3130    2749    3290
  181063] [0.11921013 0.19536062 0.27151111 0.3476616  0.42381208 0.49996257
 0.57611306 0.65226355 0.72841404 0.80456452 0.88071501]


In [206]:
print('1: {}'.format(vod(vol.get_data()>0.5, vol.get_data()>0.5, verbose=True)))

intersection	 0
union		 645376
1: 0.9999984505180692


In [209]:
print('1: {}'.format(vod(truth.get_data(), postprocess_prediction(pred.get_data(), gaussian_std=0.1, threshold=0.1), verbose=True)))
#print('2: {}'.format(vod(truth.get_data(), postprocess_prediction(pred2.get_data(), gaussian_std=1, threshold=0.5))))
#print('3: {}'.format(vod(truth.get_data(), postprocess_prediction(pred3.get_data(), gaussian_std=1, threshold=0.5))))

intersection	 295947
union		 338601
1: 0.1259709038930662


In [77]:
new_img_like(truth, data=1*truth.get_data()+1*(postprocess_prediction(pred.get_data(), gaussian_std=0.5, threshold=0.5).astype(int))).orthoview()

<OrthoSlicer3D: (256, 256, 52)>

Traceback (most recent call last):
  File "/home/galdude33/anaconda3/envs/keras/lib/python3.6/site-packages/matplotlib/cbook/__init__.py", line 388, in process
    proxy(*args, **kwargs)
  File "/home/galdude33/anaconda3/envs/keras/lib/python3.6/site-packages/matplotlib/cbook/__init__.py", line 228, in __call__
    return mtd(*args, **kwargs)
TypeError: _cleanup() takes 1 positional argument but 2 were given


In [218]:
for i in range(10):
    print('{:.2} - {:.8}'.format(i/3, vod(truth.get_data(), gaussian_filter(pred.get_data(), i/3) > 0.5)))

0.0 - 0.14122965
0.33 - 0.14118736
0.67 - 0.14025291
1.0 - 0.15213494
1.3 - 0.1559348
1.7 - 0.15859626
2.0 - 0.16103441
2.3 - 0.16361388
2.7 - 0.16607572
3.0 - 0.16784486


In [220]:
for i in range(0,21):
    threshold = i * 5 / 100
    print('{} - {}'.format(threshold, vod(truth.get_data(), gaussian_filter(pred.get_data(), 0.33) > threshold)))

0.0 - 0.9065614372518576
0.05 - 0.14814359689532586
0.1 - 0.13409773457672813
0.15 - 0.1278775639714016
0.2 - 0.12535605723218368
0.25 - 0.12583704184158218
0.3 - 0.12729425517299553
0.35 - 0.12959742000942132
0.4 - 0.1326181465043882
0.45 - 0.13660837921286184
0.5 - 0.14120474323651855
0.55 - 0.14619835062322617
0.6 - 0.1525275180080523
0.65 - 0.15985079129981627
0.7 - 0.16676434855339228
0.75 - 0.1763829533391864
0.8 - 0.18880411182394718
0.85 - 0.20404091332511465
0.9 - 0.2253708992666924
0.95 - 0.2582048951764738
1.0 - 0.9999968056833283


In [112]:
for i in range(0,1):
    print('{} - {}'.format(i, vod(truth.get_data(), sobel(pred.get_data()) > 0.4)))

0 - 0.8936488796427577


In [37]:
from scipy.ndimage.morphology import binary_fill_holes
vod(truth.get_data(), binary_fill_holes(gaussian_filter(pred.get_data(), 1) > 0.5))

0.5230235351692842

In [10]:
vod(truth.get_data(), postprocess_prediction(pred.get_data(), gaussian_std=1, threshold=0.5, connected_component=True), verbose=True)

intersection	 175895
union		 196316


0.10402053821115853

In [14]:
np.histogram(pred.get_data())

(array([3125052,   11203,    7935,    8868,  112831,   40350,    2922,
           2352,    2914,  588549]),
 array([0.11920303, 0.19536243, 0.27152183, 0.34768123, 0.42384063,
        0.50000003, 0.57615943, 0.65231883, 0.72847823, 0.80463763,
        0.88079703]))

In [178]:
rep1 = (vol.get_data())
rep2 = postprocess_prediction(pred.get_data(), fill_holes=True, gaussian_std=0.5, threshold=0.5)
rep3 = (truth.get_data())
rep = np.concatenate([np.expand_dims(rep1+3*rep2, -1), np.expand_dims(rep1+3*rep3, -1)], axis=-1)
new_img_like(vol, data=rep).orthoview()

/home/galdude33/anaconda3/envs/keras/lib/python3.6/site-packages/nibabel/viewers.py:416: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  vdata = self._data[idx].ravel()


<OrthoSlicer3D: (256, 256, 108, 2)>

Traceback (most recent call last):
  File "/home/galdude33/anaconda3/envs/keras/lib/python3.6/site-packages/matplotlib/cbook/__init__.py", line 388, in process
    proxy(*args, **kwargs)
  File "/home/galdude33/anaconda3/envs/keras/lib/python3.6/site-packages/matplotlib/cbook/__init__.py", line 228, in __call__
    return mtd(*args, **kwargs)
TypeError: _cleanup() takes 1 positional argument but 2 were given
